In [ ]:
# Import libraries
import csv
from ctypes import Array
from types import new_class
from bs4 import BeautifulSoup, NavigableString
import re
import json

In [ ]:
def parsing(file):

    # Reading the data inside the xml file
    with open(file, 'r', encoding="utf8") as f:
        data = f.read()

    # Passing the stored data inside the beautifulsoup parser, storing the returned object
    Bs_data = BeautifulSoup(data, "lxml")

    # Extrcting all sections
    main_div = Bs_data.find('div', attrs = {'class':"reboot collapse show"})
    sections = main_div.find_all('li')
    temp = []
    for i in range (len(sections)):
        item = sections[i]
        ss = re.findall(">S[0-9]+.*<",str(item.encode("utf-8")))
        if (len(ss) > 0):
            temp.append(ss[0][1:11])
    sections = temp

    divvv = Bs_data.find('div', attrs = {'id':"dynamic-xbrl-form"})
    all_div = divvv.find_all('div')

    all_div.pop(0)

    new_arr = []
    z = 0
    for sec in sections:
        for dv in all_div:

            item = str(dv)
            if sec in item:
                try:
                    dv_first_sibling = dv.next_element.next_element
                except:
                    continue
                while((sec not in str(dv_first_sibling))):
                    try:
                        dv_first_sibling = dv_first_sibling.next_sibling.next_sibling
                    except:
                        break
                try:
                    target_div = dv_first_sibling.next_element.next_element.next_element.next_element.next_element.next_element.next_element.next_element.next_sibling.next_sibling.next_element.next_element
                except:
                    continue
                useful_text = ""
                arr = []
                while(target_div and not isinstance(target_div, NavigableString)):
                    temp_arr = target_div.find_all('table')
                    if (len(temp_arr) > 0):
                        arr.append(useful_text)
                        useful_text = ""

                        # table
                        rows = target_div.find_all('tr')
                        cols = rows[0].find_all('td')
                        mat = []
                        temp = []
                        for i in range(len(rows)):
                            if (i == 0):
                                for j in rows[i].find_all('td'):
                                    tx = ""
                                    for k in j.find_all('span'):
                                        tx = tx + " " + k.text
                                    temp.append(tx)
                            else:
                                extra = ""
                                for idx, j in enumerate(rows[i].find_all('td')):
                                    tx = ""
                                    if (idx == 0):  
                                        k = j.find('span')
                                        if (k == None):
                                            continue
                                        tx = tx + " " + k.text
                                        tg = k.find("ix:nonfraction")
                                        if (tg):
                                            tg = tg["name"][3:]
                                        else:
                                            tg = ""
                                            
                                        extra = tx
                                    else:
                                        k = j.find('span')
                                        if (k == None):
                                            continue
                                        tx = tx + " " + k.text
                                        tg = k.find("ix:nonfraction")
                                        if (tg):
                                            tg = tg["name"][3:]
                                        else:
                                            tg = ""

                                        obj = {
                                            "row" : extra,
                                            "col" : temp[idx],
                                            "data" : tx,
                                            "tag" : tg
                                        }
                                        mat.append(obj)
                        arr.append(mat)
                    elif (target_div):
                        spans = target_div.find_all('span')
                        for sp in spans:
                            useful_text = useful_text + " " + sp.text

                    if (target_div and target_div.next_sibling !=None):
                        target_div = target_div.next_sibling.next_sibling
                    else:
                        break
                arr.append(useful_text)
                for i in range(len(arr)):
                    if (type(arr[i]) == list):
                        if (i > 0 and i < len(arr) - 1):
                            obj = {
                                "Above_txt" : arr[i-1],
                                "Table_data" : arr[i],
                                "Below_txt" : arr[i+1]
                            }
                            new_arr.append(obj)
                        elif (i > 0 and i == len(arr) - 1):
                            obj = {
                                "Above_txt" : arr[i-1],
                                "Table_data" : arr[i],
                                "Below_txt" : ""
                            }
                            new_arr.append(obj)
                        elif (i == 0 and i < len(arr) - 1):
                            obj = {
                                "Above_txt" : "",
                                "Table_data" : arr[i],
                                "Below_txt" : arr[i+1]
                            }
                            new_arr.append(obj)

    
    # Writing to sample.json
    with open("sample.json", "w") as outfile:
            # Serializing json
            json_object = json.dumps(new_arr, indent=4)
            outfile.write(json_object)


In [ ]:
def json_to_csv(file):

    # Opening JSON file
    with open(file, 'r') as openfile:
    
        # Reading from json file
        json_object = json.load(openfile)

    final = []
    for i in range(len(json_object)):
        above = json_object[i]["Above_txt"]
        below = json_object[i]["Below_txt"]
        arr = json_object[i]["Table_data"]
        for j in range(len(arr)):
            row = arr[j]["row"]
            col = arr[j]["col"]
            data = arr[j]["data"]
            if (data ==  " None"):
                data = ""
            tag = arr[j]["tag"]
            final.append({"Above Text" : above, "Row Text": row, "Col Text": col, "Data": data, "Below Text": below, "Tag" : tag})

    # opening the csv file in 'w' mode
    file = open('data.csv', 'w',  encoding="utf-8", newline ='')

    with file:
        # identifying header 
        header = ["Above Text", "Row Text", "Col Text", "Data", "Below Text", "Tag"]
        writer = csv.DictWriter(file, fieldnames = header)
        
        # writing data row-wise into the csv file
        writer.writeheader()
        writer.writerows(final)

In [ ]:
# Call function "parsing" by passing "file.xml" and get sample.json file
parsing("file.xml")

# convert json to csv
json_to_csv("sample.json")